In [ ]:
import discord
from discord.ext import tasks, commands
from datetime import datetime
import requests
import json
import time
import bs4
import nest_asyncio
nest_asyncio.apply()

file = open("token.txt", mode="r")
token = file.readline().strip()
general_channel = int(file.readline().strip())

In [ ]:
class Coin:
    def __init__(self, symbol):
        self.symbol = symbol.upper()
        self.price = self.coin_price()
        self.is_valid = True
        
    def coin_price(self):
        try:
            unix_time = round(time.time()*1000)
            url = "https://http-api.livecoinwatch.com/coins/history/range?coin=" + self.symbol + "&start=" + str(unix_time-500000) + "&end=" + str(unix_time) + "&currency=USD"
            req = requests.get(url).text
            soup = bs4.BeautifulSoup(req)

            d = json.loads(soup.body.text)
            price = d["data"][-1]["rate"]
            return round(price, 2)
        except:
            self.is_valid = False

    def print_price(self):
        st = ""
        current_price = self.coin_price()
        
        if not self.is_valid:
            return
        
        st += self.symbol + ": \n$" + str(current_price) + " (" + str(round(100*(current_price-self.price)/self.price, 2)) + "%) "
        st += ("🚀" if current_price > self.price else "📉" if current_price < self.price else "") + "\n\n"
        
        self.price = current_price
        return st

In [ ]:
client = commands.Bot(command_prefix = ".")
coins = []
coin_symbols = set()

@client.event
async def on_ready():
    channel = client.get_channel(general_channel)
    print_prices.start()
    print("Bot initialized")
    
    await channel.send("**Hello! Type !help for more information**")
    
@client.event
async def on_message(message):
    if "!" in message.content and message.content[0] == "!":
        if message.content.lower().strip() == "!help":
            st = "**INSTRUCTIONS**\n```"
            st += "- Use !<symbol> to add that respective coin (e.g. BTC for Bitcoin, LINK for ChainLink, etc).\n"
            st += "- Use !remove <symbol> to remove coin from active list.\n"
            st += "- Type !coins to view active tokens and prices."
            st += "```"
            await message.channel.send(st)
        elif message.content.lower().strip() == "!coins":
            if coins:
                st = "```" + str(len(coins)) + " active coins:\n\n"
                
                for i in range(len(coins)):
                    coins[i].price = coins[i].coin_price()
                
                for i,j in enumerate(coins):
                    st += str(i+1) + ". " + j.symbol + " ($" + str(j.price) + ")" + "\n"
                st += "```"
            else:
                st = "```No active coins. Add one now!```"
                
            await message.channel.send(st)
        elif "!remove" in message.content.lower().strip():
            if coins:
                to_remove = message.content.lower().strip()[message.content.lower().strip().index("!remove")+7:].strip().upper()
                if to_remove in coin_symbols:
                    coin_symbols.remove(to_remove)
                    for i in range(len(coins)):
                        if coins[i].symbol == to_remove:
                            coins.pop(i)
                            break

                    await message.channel.send("```Successfully removed coin: " + to_remove + " ✔```")        

                else:
                    await message.channel.send("```Could not find coin: " + to_remove + " in active list. ❌```" )
            else:
                await message.channel.send("```No active coins to remove. ❌```")
            
        else:
            symbol = message.content[1:].strip().upper()
            new_coin = Coin(symbol)
            st = new_coin.print_price()

            if not new_coin.is_valid:
                await message.channel.send("```Unable to find data for coin: " + symbol + " ❌```")
            else:
                if symbol not in coin_symbols:
                    coin_symbols.add(symbol)
                    coins.append(new_coin)
                    await message.channel.send("```Successfully added coin: " + symbol + " ✔```")

                else:
                    await message.channel.send("```" + symbol + " is already added!```")

@tasks.loop(seconds=600)
async def print_prices():
    channel = client.get_channel(general_channel)
    
    if coins:
        message = "**⌚ PRICE UPDATE: *" + datetime.now().strftime("%H:%M:%S") + " EST***\n```"
        for coin in coins:
            message += coin.print_price()

        message += "```"
        await channel.send(message)
        
client.run(token)